In [26]:
import duckdb
import pyarrow.parquet as pq
import s3fs
import time

# Acceso a parquets particionados en S3

In [2]:
fs = s3fs.S3FileSystem(anon=False, endpoint_url="http://localhost:9000",
                 key='catedra', secret='catedrapass', use_ssl=False)

# Acceso a parquet particula
Se puede leer un parquet individual con pyarrow, o usar filtros para resolver consultas 

In [3]:
pq_table = pq.read_table('randata/month=6/day=1/0.parquet', filesystem=fs)
pq_table

pyarrow.Table
ts: date32[day]
cantidad: int32
hour: int32
valor: float
tipo: string
----
ts: [[2023-06-01,2023-06-01,2023-06-01,2023-06-01,2023-06-01,...,2023-06-01,2023-06-01,2023-06-01,2023-06-01,2023-06-01]]
cantidad: [[45,87,75,13,10,...,19,26,75,97,62]]
hour: [[0,0,0,0,0,...,23,23,23,23,23]]
valor: [[0.79934084,0.4796671,0.13389292,0.8573266,0.14717335,...,0.2803504,0.88382137,0.7066165,0.4143071,0.23564212]]
tipo: [["bbb","bbb","bbb","aaa","ddd",...,"ddd","ddd","aaa","ccc","aaa"]]

In [9]:
pq_table_query = pq.read_table('randata/',
                        filters=[('valor','>',0.99),('tipo','=','aaa')], filesystem=fs)
pq_table_query.to_pandas().head()

,ts,cantidad,hour,valor,tipo,month,day
0,2023-05-28,8,2,0.994326,aaa,5,28
1,2023-05-28,28,3,0.998410,aaa,5,28
2,2023-05-28,33,6,0.991118,aaa,5,28
3,2023-05-28,41,7,0.995455,aaa,5,28
4,2023-05-28,57,8,0.998239,aaa,5,28


# Acceso a un dataset
Pyarrow tambien permite el acceso a [Dataset](https://arrow.apache.org/docs/python/generated/pyarrow.dataset.Dataset.html) un conjunto de parquets particionado.

En minio podemos ver la estructura del particionado, que sigue el formato de hive para los nombres de los paths.

Un dataset tambien soporta filtros para hacer consultas sobre todos los datos.

In [10]:
dataset = pq.ParquetDataset('randata/', filesystem=fs)
dataset.files[:5]

['randata/month=5/day=28/0.parquet',
 'randata/month=5/day=29/0.parquet',
 'randata/month=5/day=30/0.parquet',
 'randata/month=5/day=31/0.parquet',
 'randata/month=6/day=1/0.parquet']

In [17]:
dataset = pq.ParquetDataset('randata/',
                        filters=[('valor','>',0.99),('tipo','=','aaa')], filesystem=fs)
result_df = dataset.read_pandas().to_pandas()
result_df.head()

,ts,cantidad,hour,valor,tipo,month,day
0,2023-05-28,8,2,0.994326,aaa,5,28
1,2023-05-28,28,3,0.998410,aaa,5,28
2,2023-05-28,33,6,0.991118,aaa,5,28
3,2023-05-28,41,7,0.995455,aaa,5,28
4,2023-05-28,57,8,0.998239,aaa,5,28


In [18]:
len(result_df)

93

# Aceso a los parquets como una db analitica

[Duckdb](https://duckdb.org/) es una db analitica en memoria que permite acceder a un conjunto de parquets particionados y hacer consultas de manera eficiente.

In [19]:
duckdb.sql("""INSTALL httpfs;
LOAD httpfs;
SET s3_region='us-east-1';
SET s3_url_style='path';
SET s3_endpoint='localhost:9000';
SET s3_access_key_id='catedra' ;
SET s3_use_ssl=false;
SET s3_secret_access_key='catedrapass';""")

Podemos acceder a un solo archivo y resolver las consultas que teniamos en el notebook de serializacion

In [30]:
start_time = time.time()
result = duckdb.sql("""
    SELECT * 
    FROM parquet_scan('s3://flujovehicular/flujo.parquet')
    WHERE CANTIDAD > 13000
        AND SENTIDO = 'Egreso';
""")
duckdb_time = time.time()-start_time
result

┌─────────────────┬─────────────────────┬──────────┬─────────┬──────────┬────────────┐
│ CODIGO_LOCACION │        HORA         │ CANTIDAD │ SENTIDO │ LATITUD  │  LONGITUD  │
│     varchar     │      timestamp      │  int32   │ varchar │  float   │   float    │
├─────────────────┼─────────────────────┼──────────┼─────────┼──────────┼────────────┤
│ 48Q39G00+       │ 2020-03-10 17:00:00 │    13041 │ Egreso  │ -34.6333 │ -58.468594 │
│ 48Q39G00+       │ 2020-03-06 17:00:00 │    13041 │ Egreso  │ -34.6333 │ -58.468594 │
└─────────────────┴─────────────────────┴──────────┴─────────┴──────────┴────────────┘

In [31]:
duckdb_time

0.008410930633544922

In [32]:
start_time = time.time()
result = duckdb.sql("""
    SELECT * 
    FROM parquet_scan('s3://flujovehicular/flujo.parquet')
    WHERE HORA > '2020-03-10 00:00:00' AND HORA < '2020-03-12 00:00:00';
""")
duckdb_time = time.time()-start_time
result

┌─────────────────┬─────────────────────┬──────────┬─────────┬────────────┬────────────┐
│ CODIGO_LOCACION │        HORA         │ CANTIDAD │ SENTIDO │  LATITUD   │  LONGITUD  │
│     varchar     │      timestamp      │  int32   │ varchar │   float    │   float    │
├─────────────────┼─────────────────────┼──────────┼─────────┼────────────┼────────────┤
│ 48Q3CH00+       │ 2020-03-11 23:00:00 │     1385 │ Interna │  -34.57963 │ -58.427734 │
│ 48Q3FG00+       │ 2020-03-11 23:00:00 │      442 │ Interna │  -34.53698 │ -58.470966 │
│ 48Q3CJ00+       │ 2020-03-11 23:00:00 │     2381 │ Interna │ -34.588795 │  -58.38152 │
│ 48Q3CJ00+       │ 2020-03-11 23:00:00 │     1431 │ Ingreso │ -34.588795 │  -58.38152 │
│ 48Q3CJ00+       │ 2020-03-11 23:00:00 │     1478 │ Egreso  │ -34.588795 │  -58.38152 │
│ 48Q3CH00+       │ 2020-03-11 23:00:00 │      117 │ Egreso  │  -34.57963 │ -58.427734 │
│ 48Q39J00+       │ 2020-03-11 23:00:00 │     3181 │ Interna │  -34.60855 │  -58.37295 │
│ 48Q39H00+       │ 2

In [33]:
duckdb_time

0.008717060089111328

# Crear una db en memoria
Se pueden traer todos los datos del dataset y hacer consultas directamente en memoria.

In [34]:
duckdb.sql("""
    CREATE TABLE randat_mem AS
    SELECT * 
    FROM parquet_scan('s3://randata/*/*/*.parquet', hive_partitioning=1);
""")

In [35]:
duckdb.sql("""
    SELECT count(*) from randat_mem WHERE valor>0.99;
""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          327 │
└──────────────┘

In [43]:
duckdb.sql("""
    SELECT * 
    FROM randat_mem
    WHERE valor > 0.99 AND tipo = 'bbb';
""")

┌────────────┬──────────┬───────┬────────────┬─────────┬───────┬───────┐
│     ts     │ cantidad │ hour  │   valor    │  tipo   │  day  │ month │
│    date    │  int32   │ int32 │   float    │ varchar │ int64 │ int64 │
├────────────┼──────────┼───────┼────────────┼─────────┼───────┼───────┤
│ 2023-05-28 │       32 │     1 │ 0.99896485 │ bbb     │    28 │     5 │
│ 2023-05-28 │       58 │     2 │  0.9984177 │ bbb     │    28 │     5 │
│ 2023-05-28 │       17 │    13 │  0.9906464 │ bbb     │    28 │     5 │
│ 2023-05-28 │       48 │    14 │  0.9926076 │ bbb     │    28 │     5 │
│ 2023-05-28 │       73 │    17 │   0.998972 │ bbb     │    28 │     5 │
│ 2023-05-28 │       35 │    21 │  0.9903448 │ bbb     │    28 │     5 │
│ 2023-05-28 │       45 │    21 │  0.9901861 │ bbb     │    28 │     5 │
│ 2023-05-28 │       28 │    21 │  0.9922164 │ bbb     │    28 │     5 │
│ 2023-05-28 │       24 │    22 │   0.993563 │ bbb     │    28 │     5 │
│ 2023-05-29 │       49 │     0 │  0.9903087 │ bbb 

# Nuevos datos
Si estamos en un escenario donde entran nuevos datos, la db en memoria puede quedar desactualizada, y hacer un escaneo directo de los parquets aunque es mas lento va a tener datos mas frescos.

In [45]:
duckdb.sql("""
    SELECT * 
    FROM parquet_scan('s3://randata/*/*/*.parquet', hive_partitioning=1)
    WHERE valor > 0.99 AND tipo = 'bbb';
""")

┌────────────┬──────────┬───────┬────────────┬─────────┬───────┬───────┐
│     ts     │ cantidad │ hour  │   valor    │  tipo   │  day  │ month │
│    date    │  int32   │ int32 │   float    │ varchar │ int64 │ int64 │
├────────────┼──────────┼───────┼────────────┼─────────┼───────┼───────┤
│ 2023-05-28 │       32 │     1 │ 0.99896485 │ bbb     │    28 │     5 │
│ 2023-05-28 │       58 │     2 │  0.9984177 │ bbb     │    28 │     5 │
│ 2023-05-28 │       17 │    13 │  0.9906464 │ bbb     │    28 │     5 │
│ 2023-05-28 │       48 │    14 │  0.9926076 │ bbb     │    28 │     5 │
│ 2023-05-28 │       73 │    17 │   0.998972 │ bbb     │    28 │     5 │
│ 2023-05-28 │       35 │    21 │  0.9903448 │ bbb     │    28 │     5 │
│ 2023-05-28 │       45 │    21 │  0.9901861 │ bbb     │    28 │     5 │
│ 2023-05-28 │       28 │    21 │  0.9922164 │ bbb     │    28 │     5 │
│ 2023-05-28 │       24 │    22 │   0.993563 │ bbb     │    28 │     5 │
│ 2023-05-29 │       49 │     0 │  0.9903087 │ bbb 